In [1]:
# Some of the code refered from: https://www.kaggle.com/svpons/feature-engineering

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# session

In [3]:
df_session = pd.read_csv('data/sessions.csv')

In [5]:
df_session.head(10)

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,jpmpj40cvk,lookup,NaN,NaN,Windows Desktop,957.0
5,jpmpj40cvk,search_results,click,view_search_results,Windows Desktop,965.0
6,jpmpj40cvk,search_results,click,view_search_results,Windows Desktop,1021.0
7,jpmpj40cvk,search_results,click,view_search_results,Windows Desktop,938.0
8,jpmpj40cvk,search_results,click,view_search_results,Windows Desktop,2702.0
9,jpmpj40cvk,ajax_refresh_subtotal,click,change_trip_characteristics,Windows Desktop,674.0


In [20]:
df_session.isnull().sum()

user_id          2797
action              0
action_type         0
action_detail       0
device_type         0
secs_elapsed        0
dtype: int64

In [19]:
df_session.action = df_session.action.fillna("NaN")
df_session.action_type = df_session.action_type.fillna("NaN")
df_session.action_detail = df_session.action_detail.fillna("NaN")
df_session.secs_elapsed = df_session.secs_elapsed.fillna("NaN")

In [21]:
df_session.action.value_counts()

show                               209655
search_results                     109430
personalize                        105718
lookup                              88302
ajax_refresh_subtotal               71482
index                               69441
similar_listings                    53814
update                              38829
search                              26520
header_userpic                      17332
create                              17262
track_page_view                     17089
active                              15855
social_connections                  15301
collections                         12763
dashboard                            9535
calendar_tab_inner2                  9353
ajax_check_dates                     8387
NaN                                  7873
edit                                 7716
confirm_email                        7424
ask_question                         6994
requested                            6242
authenticate                      

In [ ]:
# action values with low frequency are changed to "Other"

act_freq = 100